# Drift Detection Under Delayed Labels: Demo

This notebook demonstrates the geometric drift detection system for distinguishing label shift from concept drift.

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

from labelshift_drift.simulation.data_generators import ScenarioGenerator, apply_score_transform
from labelshift_drift.reference.fit_reference import fit_reference_model
from labelshift_drift.detector.thresholds import calibrate_thresholds
from labelshift_drift.detector.drift_detector import DriftDetector
from labelshift_drift.viz.plots import plot_drift_detection_summary, plot_state_distribution

%matplotlib inline
plt.rcParams['figure.figsize'] = (15, 10)

## Configuration

In [ ]:
# Experiment settings
SEED = 42
np.random.seed(SEED)

# Data generation
n_ref_train = 50000
n_ref_val = 20000
T_deploy = 200000
d = 10
delta = 2.0
pi_ref = 0.02

# Detector
n_u = 2000
tau0 = 0.5
lambda_ewma = 0.1

# Calibration
N_cal = 200
alpha_d = 0.01

## Setup: Generate Reference Data and Train Model

In [ ]:
# Initialize generator
generator = ScenarioGenerator(d=d, delta=delta, pi_ref=pi_ref, seed=SEED)

# Generate reference data
print("Generating reference data...")
(X_train, Y_train), (X_val, Y_val) = generator.generate_reference(n_ref_train, n_ref_val)

print(f"Training set: {X_train.shape}, π = {Y_train.mean():.4f}")
print(f"Validation set: {X_val.shape}, π = {Y_val.mean():.4f}")

In [ ]:
# Train model
print("Training logistic regression...")
model = LogisticRegression(max_iter=1000, random_state=SEED)
model.fit(X_train, Y_train)

# Get scores
S_train = model.predict_proba(X_train)[:, 1]
S_val = model.predict_proba(X_val)[:, 1]

print(f"Training AUC: {roc_auc_score(Y_train, S_train):.4f}")
print(f"Validation AUC: {roc_auc_score(Y_val, S_val):.4f}")

## Fit Reference Model and Calibrate Thresholds

In [ ]:
# Fit reference model
print("Fitting reference model...")
ref_model = fit_reference_model(S_val, Y_val, tau0=tau0, lambda_ewma=lambda_ewma, bootstrap_seed=SEED)

print(f"π_ref = {ref_model.pi_ref:.4f}")
print(f"det(C) = {np.linalg.det(ref_model.C_hat):.4f}")
print(f"δ_C = {ref_model.delta_C:.4f}")
print(f"\nConfusion matrix C:")
print(ref_model.C_hat)

In [ ]:
# Calibrate thresholds
print("Calibrating thresholds...")
ref_model = calibrate_thresholds(ref_model, S_val, Y_val, n_u=n_u, N_cal=N_cal, alpha_d=alpha_d, seed=SEED)

print(f"d_th = {ref_model.d_th:.4f}")
print(f"r_th = {ref_model.r_th:.4f}" if ref_model.r_th else "r_th = None")
print(f"π_th = {ref_model.pi_th:.4f}")

## Scenario 1: Pure Label Shift (Valid Adaptation)

In [ ]:
print("Scenario 1: Pure Label Shift")
X_deploy, Y_deploy, pi_true = generator.scenario_1_pure_label_shift(T_deploy)
S_deploy = model.predict_proba(X_deploy)[:, 1]

print(f"Deployment samples: {len(S_deploy)}")
print(f"Deployment π: {Y_deploy.mean():.4f}")

In [ ]:
# Run detector
detector_s1 = DriftDetector(ref_model, n_u=n_u, step_u=n_u)
df_s1 = detector_s1.process_stream(S_deploy)

print(f"\nProcessed {len(df_s1)} windows")
print(f"\nState distribution:")
print(df_s1['state'].value_counts())

In [ ]:
# Visualize
plot_drift_detection_summary(df_s1, ref_model, pi_true=pi_true, save_path='../artifacts/figures/scenario_1_demo.png')

In [ ]:
plot_state_distribution(df_s1, save_path='../artifacts/figures/scenario_1_states.png')

## Scenario 2: Concept Drift (Invalid for Adaptation)

In [ ]:
print("Scenario 2: Concept Drift")
X_deploy, Y_deploy, drift_indicator = generator.scenario_2_concept_drift(T_deploy, drift_time=80000)
S_deploy = model.predict_proba(X_deploy)[:, 1]

print(f"Deployment samples: {len(S_deploy)}")
print(f"Drift occurs at t=80000")

In [ ]:
# Run detector
detector_s2 = DriftDetector(ref_model, n_u=n_u, step_u=n_u)
df_s2 = detector_s2.process_stream(S_deploy)

print(f"\nProcessed {len(df_s2)} windows")
print(f"\nState distribution:")
print(df_s2['state'].value_counts())

In [ ]:
# Visualize
plot_drift_detection_summary(df_s2, ref_model, drift_indicator=drift_indicator, save_path='../artifacts/figures/scenario_2_demo.png')

## Scenario 4: Covariate Shift (Benign - Should NOT Alarm)

In [ ]:
print("Scenario 4: Covariate Shift (Benign)")
X_deploy, Y_deploy, shift_indicator = generator.scenario_4_covariate_shift_benign(T_deploy, drift_time=80000)
S_deploy = model.predict_proba(X_deploy)[:, 1]

print(f"Deployment samples: {len(S_deploy)}")
print(f"Covariate shift at t=80000 (in unused dimension)")

In [ ]:
# Run detector
detector_s4 = DriftDetector(ref_model, n_u=n_u, step_u=n_u)
df_s4 = detector_s4.process_stream(S_deploy)

print(f"\nProcessed {len(df_s4)} windows")
print(f"\nState distribution:")
print(df_s4['state'].value_counts())

In [ ]:
# Visualize
plot_drift_detection_summary(df_s4, ref_model, drift_indicator=shift_indicator, save_path='../artifacts/figures/scenario_4_demo.png')

## Summary

This notebook demonstrated the complete drift detection system:

1. **Scenario 1 (Pure Label Shift)**: The detector correctly identifies prevalence changes and enters PRIOR_SHIFT state, adapting the operational threshold while maintaining geometric consistency.

2. **Scenario 2 (Concept Drift)**: The detector correctly identifies geometric violations (mixture consistency failures) and enters DRIFT_SUSPECTED state, freezing adaptation and signaling the need for retraining.

3. **Scenario 4 (Benign Covariate Shift)**: The detector correctly does NOT alarm for covariate shifts in dimensions orthogonal to the decision boundary.

The system successfully distinguishes label shift (safe for threshold adaptation) from concept drift (requires retraining).